# Antenna response to galactic radio emission

In [ ]:
import os
import sys
from radiocalibrationtoolkit import *
from pygdsm import GlobalSkyModel2016

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

%config InlineBackend.print_figure_kwargs = {'bbox_inches':None}

In [ ]:
# some global plot settings
plt.rcParams["axes.labelweight"] = "bold"
plt.rcParams["font.weight"] = "bold"
plt.rcParams['font.size'] = 16
plt.rcParams['legend.fontsize']= 14

plt.rcParams['xtick.major.width']= 2
plt.rcParams['ytick.major.width']= 2

plt.rcParams['xtick.major.size']= 5
plt.rcParams['ytick.major.size']= 5

plt.rcParams['xtick.labelsize']= 14
plt.rcParams['ytick.labelsize']= 14

In [ ]:
# this will be further on constant
frequency_MHz = 45 
LATITUDE = -35.206667

In [ ]:
hw_file_path = './antenna_setup_files/HardwareProfileList_realistic.xml'
hw_dict = read_hw_file(hw_file_path)

In [ ]:
hw_dict

In [ ]:
# calculate power received by the antenna
antenna_inst = AntennaPattern("./antenna_setup_files/SALLA_EW.xml")
galactic_map_inst = GlobalSkyModel2016(freq_unit="MHz")

impedance_func = hw_dict['IImpedance']['antenna_EW']

lst_range = range(24)

update_antenna_conventions = {
    'shift_phi': -90,
    'flip_theta': True,
    'flip_phi': False,
    'in_degrees': True,
    'add_invisible_sky': True
}

power_density_DF = calculate_power_spectral_density(
    antenna_inst=antenna_inst,
    galactic_map_inst=galactic_map_inst,
    lst_range=lst_range,
    freq_Mhz_range=[frequency_MHz, frequency_MHz + 1],
    latitude=LATITUDE,
    update_antenna_conventions=update_antenna_conventions,
    impedance_func=impedance_func)

power_DF = integrate_spectral_density(
    power_density_DF, integrated_MHz_bands=power_density_DF.columns.values)
power_DF.iloc[:, :] = power_DF.values * 1e12

In [ ]:
# create figures: antenna gain in gal. coordinates, galactic radio emission, antenna response to galactic emission

galactic_map = hp.ma(
            hp.pixelfunc.ud_grade(galactic_map_inst.generate(frequency_MHz), NSIDE)
        ).copy()
image_list = []

lst_range = range(24)

for lst in lst_range:
    rotation_parameters = create_rotation_parameters(lst, LATITUDE)
    rotator = Rotator(coord=['G','C'], rot=rotation_parameters)

    antenna_map_inst = antenna_inst.convert2hp(frequency=frequency_MHz, quantity='absolute', **update_antenna_conventions)
    antenna_map = antenna_map_inst.get_map(rotator=rotator)
    mask = create_local_mask(NSIDE, rotation_parameters)
    # Test
    galactic_map.mask = mask
    cmap = 'jet'

    projview(
        antenna_map**2,
        # norm="log",
        # coord=["G",'C'],
        # rot=False,
        graticule=True,
        graticule_labels=True,
        unit="VEL [$m^2$]",
        xlabel="RA",
        ylabel="DEC",
        cb_orientation="vertical",
        min=0,
        max=3,
        # max=550000,
        latitude_grid_spacing=30,
        xtick_label_color="white",
        title='Antenna gain',
        cmap=cmap,
        sub=(2, 2, 1),
        # projection_type='cart'
    )

    projview(
        galactic_map,
        norm="log",
        # coord=["G",'C'],
        # rot=False,
        graticule=True,
        graticule_labels=True,
        unit="Temperature [K]",
        xlabel="RA",
        ylabel="DEC",
        cb_orientation="vertical",
        min=3500,
        max=30000,
        # max=550000,
        latitude_grid_spacing=30,
        xtick_label_color="white",
        title='Galactic radio emission',
        cmap=cmap,
        sub=(2, 2, 2),
        # projection_type='cart'
    )

    projview(
        galactic_map*antenna_map**2,
        norm="log",
        # coord=["G",'C'],
        # rot=False,
        graticule=True,
        graticule_labels=True,
        unit="[$m^2$K]",
        xlabel="RA",
        ylabel="DEC",
        cb_orientation="vertical",
        min=3500,
        # max=30000,
        max=25000,
        latitude_grid_spacing=30,
        xtick_label_color="white",
        title='Antenna response to gal.emission',
        cmap=cmap,
        sub=(2, 2, 3),
        # projection_type='cart'
    )

    fig = plt.gcf()
    fig.autolayout=False
    ax = fig.add_subplot(2,2,4)
    ax.plot(power_DF.index.values, power_DF[frequency_MHz].values)
    ax.scatter(lst, power_DF.loc[lst, frequency_MHz], color='red', s=75)
    ax.set_title('Integrated sky by antenna')
    ax.set_xlabel('LST')
    ax.set_ylabel('power [pW]')
    fig.subplots_adjust(left=0.075, bottom=0.1, wspace=0.35)
    plt.close()
    image_list.append(fig_to_image(fig))

In [ ]:
# animate

def animate(i):
    ax.ravel()[0].imshow(image_list[i], animated=True)
    ax.ravel()[0].axis("off")
    ax.ravel()[0].set_title("\n frequency: {} MHz   LST: {:02d}:00".format(frequency_MHz, i))

fig, ax = plt.subplots(figsize=(6, 4.5), dpi=160, squeeze=False)
fig.subplots_adjust(left=0, right=1)

dynamic_figs = animation.FuncAnimation(
    fig, animate, frames=len(image_list), interval=100, repeat_delay=100
)
plt.close()

# Show the animation
HTML(dynamic_figs.to_html5_video())

# save
# dynamic_figs.save('dynamic_images.mp4')

In [ ]:
# In the local coordinate system

In [ ]:
# create figures: antenna gain in gal. coordinates, galactic radio emission, antenna response to galactic emission

image_list = []

antenna_hpmap_inst = antenna_inst.convert2hp(frequency_MHz, **update_antenna_conventions)
antenna_map = antenna_hpmap_inst.get_map()
galactic_map = hp.pixelfunc.ud_grade(galactic_map_inst.generate(frequency_MHz), 64)

for lst in lst_range:
    galactic_map_rotated = rotate_default_hpmap_array_from_galactic2local_coordinates(galactic_map, lst, LATITUDE)
    cmap = 'jet'

    projview(
        antenna_map**2,
        # norm="log",
        # coord=["G",'C'],
        # rot=False,
        graticule=True,
        graticule_labels=True,
        unit="VEL [$m^2$]",
        xlabel="azimuth",
        ylabel="altitude",
        cb_orientation="vertical",
        min=0,
        max=3,
        # max=550000,
        latitude_grid_spacing=30,
        xtick_label_color="white",
        title='Antenna gain',
        cmap=cmap,
        sub=(2, 2, 1),
        # projection_type='cart'
    )

    projview(
        galactic_map_rotated,
        norm="log",
        graticule=True,
        graticule_labels=True,
        unit="Temperature [K]",
        xlabel="azimuth",
        ylabel="altitude",
        cb_orientation="vertical",
        min=3500,
        max=30000,
        # max=550000,
        latitude_grid_spacing=30,
        xtick_label_color="white",
        title='Galactic radio emission',
        cmap=cmap,
        sub=(2, 2, 2),
        # projection_type='cart'
    )

    projview(
        galactic_map_rotated*antenna_map**2,
        norm="log",
        # coord=["G",'C'],
        # rot=False,
        graticule=True,
        graticule_labels=True,
        unit="[$m^2$K]",
        xlabel="azimuth",
        ylabel="altitude",
        cb_orientation="vertical",
        min=3500,
        # max=30000,
        max=25000,
        latitude_grid_spacing=30,
        xtick_label_color="white",
        title='Antenna response to gal.emission',
        cmap=cmap,
        sub=(2, 2, 3),
        # projection_type='cart'
    )

    fig = plt.gcf()
    fig.autolayout=False
    ax = fig.add_subplot(2,2,4)
    ax.plot(power_DF.index.values, power_DF[frequency_MHz].values)
    ax.scatter(lst, power_DF.loc[lst, frequency_MHz], color='red', s=75)
    ax.set_title('Integrated sky by antenna')
    ax.set_xlabel('LST')
    ax.set_ylabel('power [pW]')
    fig.subplots_adjust(left=0.075, bottom=0.1, wspace=0.35)
    plt.close()
    image_list.append(fig_to_image(fig))

In [ ]:
# animate

def animate(i):
    ax.ravel()[0].imshow(image_list[i], animated=True)
    ax.ravel()[0].axis("off")
    ax.ravel()[0].set_title("\n frequency: {} MHz   LST: {:02d}:00".format(frequency_MHz, i))

fig, ax = plt.subplots(figsize=(6, 4.5), dpi=160, squeeze=False)
fig.subplots_adjust(left=0, right=1)

dynamic_figs = animation.FuncAnimation(
    fig, animate, frames=len(image_list), interval=100, repeat_delay=100
)
plt.close()

# Show the animation
HTML(dynamic_figs.to_html5_video())

# save
# dynamic_figs.save('dynamic_images_local.mp4')